In [2]:
import os
import numpy as np
from scipy import stats
import mne
import matplotlib.pyplot as plt
%matplotlib qt

### List of all pure bckg files

In [1]:
## files used to generate bckg spectrograms
fileObject = open('bckg_256.txt', "r")
data = fileObject.read()
bckg_256 = data.split('\n')
bckg_256.pop()

''

In [24]:
## Channels to be used
chl = ['EEG FP1-REF', 'EEG FP2-REF',  'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF',
 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG A1-REF',
 'EEG A2-REF']

### Making Spectrograms from pure bckg files

In [31]:
plt.ioff()
save_img = "../Spectrograms/bckg/"
for path in bckg_256:
    bck = mne.io.read_raw_edf(path[:-3] + "edf", preload=True, verbose=0).pick_channels(chl)
    if bck.__len__()/bck.info['sfreq'] < 120: continue  ## skip files with duration < 120 seconds
    bck.notch_filter(freqs=np.arange(60,128,60), verbose=0)
    bck.filter(l_freq=0.5, h_freq=50, iir_params=None, method="iir", verbose = 0)
    epochs = mne.make_fixed_length_epochs(bck, duration=3, preload=True, verbose = 0)
    k=0
    for epoch in list(epochs):
        epoch_toplot = stats.trim_mean(epoch, 0.2)
        plt.figure()        #creating a plt figure object
        ax = plt.axes()     #calling axis method
        ax.set_axis_off()   #removing axis marking from the plot
        plt.specgram(epoch_toplot, Fs=256, NFFT=256, window=np.hamming(256), noverlap=192, cmap='jet')   #generating spectrogram
        fname = save_img + "bckg_" + path.split('/')[8] + "-" + path.split('/')[9][:-4] + "-spec_" + str(k) + "iter4.jpg"
        plt.savefig(fname,transparent=True,pad_inches=0,bbox_inches='tight')
        plt.close()
        k=k+1
